# Real-time Color Prediction

This notebook implements a real-time color detection system using your webcam. It will:
1. Access your webcam feed
2. Process each frame
3. Detect the color of objects in the center of the frame
4. Display the results in real-time

In [4]:
import cv2
import numpy as np
from sklearn.cluster import KMeans
import pandas as pd
import requests
import time

# Define basic colors in HSV space with broader ranges
COLORS = {
    'red': ([0, 50, 50], [10, 255, 255]),       # Red has two ranges in HSV
    'red2': ([160, 50, 50], [180, 255, 255]),   # Second red range
    'blue': ([100, 50, 50], [130, 255, 255]),
    'green': ([35, 50, 50], [85, 255, 255]),
    'yellow': ([20, 50, 50], [35, 255, 255]),
    'orange': ([10, 50, 50], [20, 255, 255]),
    'purple': ([130, 50, 50], [160, 255, 255]),
    'white': ([0, 0, 180], [180, 30, 255]),
    'black': ([0, 0, 0], [180, 255, 50])
}

In [5]:
def get_color_name(hsv_values):
    """Get color name based on HSV ranges"""
    h, s, v = hsv_values[0], hsv_values[1], hsv_values[2]
    
    for color_name, (lower, upper) in COLORS.items():
        if (lower[0] <= h <= upper[0] and
            lower[1] <= s <= upper[1] and
            lower[2] <= v <= upper[2]):
            if color_name == 'red2':
                return 'red'
            return color_name
            
    return 'unknown'

def get_dominant_color(image):
    """Get the dominant color in HSV space using K-means clustering"""
    # Convert to HSV
    hsv_image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    
    # Reshape the image
    pixels = hsv_image.reshape(-1, 3)
    
    # Use K-means to find the dominant color cluster
    kmeans = KMeans(n_clusters=1, n_init=10)
    kmeans.fit(pixels)
    
    # Get the dominant color
    dominant_color = kmeans.cluster_centers_[0].astype(int)
    
    # Ensure H value is in range 0-180 (OpenCV HSV format)
    dominant_color[0] = dominant_color[0] % 180
    
    return dominant_color

In [6]:
# Initialize the webcam
cap = cv2.VideoCapture(0)
last_api_call = time.time()

try:
    while True:
        # Read frame from webcam
        ret, frame = cap.read()
        if not ret:
            break
            
        # Get frame dimensions
        height, width = frame.shape[:2]
        
        # Define the center region (100x100 pixels)
        center_size = 100
        x1 = width//2 - center_size//2
        y1 = height//2 - center_size//2
        x2 = x1 + center_size
        y2 = y1 + center_size
        
        # Extract center region
        center_region = frame[y1:y2, x1:x2]
        
        # Get dominant color
        dominant_color = get_dominant_color(center_region)
        color_name = get_color_name(dominant_color)
        
        # Draw rectangle and color name
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, color_name, (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
        
        # Display color values
        print(f"\rDetected color: {color_name} (HSV: {dominant_color})", end="")
        
        # Send color detection to backend every 3 seconds
        current_time = time.time()
        if current_time - last_api_call >= 3:
            try:
                requests.post('http://localhost:3000/api/detections', 
                    json={
                        'color': color_name,
                        'hsv_values': dominant_color.tolist()
                    },
                    timeout=1
                )
                last_api_call = current_time
            except requests.exceptions.RequestException:
                pass  # Silently handle connection errors
        
        # Show frame
        cv2.imshow('Color Predictor', frame)
        # Break loop on 'q' press
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
            
finally:
    # Release resources
    cap.release()
    cv2.destroyAllWindows()

Detected color: green (HSV: [ 68  56 118])])